# Budget Analysis

---

## Environment Setup

In [1]:
# Initial Imports
import os
import plaid
import requests
import datetime
import json
import pandas as pd
from dotenv import load_dotenv

%matplotlib inline

In [21]:
# Helper functions

def pretty_print_response(response):
    """
    This function formats a JSON file to pretty printing it in the screen.
    """
    print(json.dumps(response, indent=4, sort_keys=True))

In [22]:
# Set environment variables from the .env file
load_dotenv()

True

In [23]:
# Extract API keys from environment variables
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SECRET = os.getenv("PLAID_SBX_SECRET_KEY")
PLAID_PUBLIC_KEY = os.getenv("PLAID_PUBLIC_KEY")

---

## Plaid Access Token

In this section, you will use the `plaid-python` API to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:

1. Create a client to connect to plaid

2. Use the client to generate a public token and request the following items: ['transactions', 'income', 'assets']

3. Exchange the public token for an access token

4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [24]:
# Create the client object
client = plaid.Client(client_id=PLAID_CLIENT_ID, secret=PLAID_SECRET, public_key=PLAID_PUBLIC_KEY, environment='sandbox')

In [25]:
# Set the institution id
INSTITUTION_ID = "ins_109508"

### 2. Generate a public token

In [26]:
# Create the public token
pub_tkn = client.Sandbox.public_token.create(INSTITUTION_ID, ['transactions','income','assets'])
# Print the public token
print(type(pub_tkn))

<class 'dict'>


### 3. Exchange the public token for an access token

In [27]:
# Retrieve the access token
# Exchange public token for access token
exchange_tkn = client.Item.public_token.exchange(pub_tkn['public_token'])
# Print the access toke
print(type(exchange_tkn))

<class 'dict'>


In [28]:
# Store the access token in a Python variable
access_tkn = exchange_tkn['access_token']

### 4. Fetch Accounts

In [29]:
# Retrieve accounts
# client.Accounts.get(access_tkn)
x = client.Accounts.get(access_tkn)
# Print accounts
pretty_print_response(x)

{
    "accounts": [
        {
            "account_id": "ZL6yG5vdMpCgGoXlaVNyHBe8Pl1XNzFglpJmB",
            "balances": {
                "available": 100,
                "current": 110,
                "iso_currency_code": "USD",
                "limit": null,
                "unofficial_currency_code": null
            },
            "mask": "0000",
            "name": "Plaid Checking",
            "official_name": "Plaid Gold Standard 0% Interest Checking",
            "subtype": "checking",
            "type": "depository"
        },
        {
            "account_id": "Mx6NB5pE4RuKGaZV4wBJh1XgVbjD6pI9yv4p5",
            "balances": {
                "available": 200,
                "current": 210,
                "iso_currency_code": "USD",
                "limit": null,
                "unofficial_currency_code": null
            },
            "mask": "1111",
            "name": "Plaid Saving",
            "official_name": "Plaid Silver Standard 0.1% Interest Saving",
       

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days

2. Print the categories for each transaction type

3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)

4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [30]:
# Set the start and end date 90 days from today
start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())
# Retrieve the transactions for the last 90 days
transaction_response = client.Transactions.get(access_tkn, start_date,end_date)
# Print the transactions
# print(json.dumps(transaction_response['transactions'][:2],indent=4, sort_keys=True))
pretty_print_response(transaction_response)

{
    "accounts": [
        {
            "account_id": "ZL6yG5vdMpCgGoXlaVNyHBe8Pl1XNzFglpJmB",
            "balances": {
                "available": 100,
                "current": 110,
                "iso_currency_code": "USD",
                "limit": null,
                "unofficial_currency_code": null
            },
            "mask": "0000",
            "name": "Plaid Checking",
            "official_name": "Plaid Gold Standard 0% Interest Checking",
            "subtype": "checking",
            "type": "depository"
        },
        {
            "account_id": "Mx6NB5pE4RuKGaZV4wBJh1XgVbjD6pI9yv4p5",
            "balances": {
                "available": 200,
                "current": 210,
                "iso_currency_code": "USD",
                "limit": null,
                "unofficial_currency_code": null
            },
            "mask": "1111",
            "name": "Plaid Saving",
            "official_name": "Plaid Silver Standard 0.1% Interest Saving",
       

### 2. Print the categories for each transaction

In [31]:
# Create a for-loop to print the categories for each transaction
    
for teacher in transaction_response['transactions']:
    print(teacher['category'])

['Payment', 'Credit Card']
['Travel', 'Taxi']
['Transfer', 'Debit']
['Transfer', 'Deposit']
['Recreation', 'Gyms and Fitness Centers']
['Travel', 'Airlines and Aviation Services']
['Food and Drink', 'Restaurants', 'Fast Food']
['Food and Drink', 'Restaurants', 'Coffee Shop']
['Food and Drink', 'Restaurants']
['Transfer', 'Credit']
['Travel', 'Airlines and Aviation Services']
['Travel', 'Taxi']
['Food and Drink', 'Restaurants']
['Payment']
['Food and Drink', 'Restaurants', 'Fast Food']
['Shops', 'Sporting Goods']


### 3. Create a new DataFrame using the following fields from the JSON transaction data: `date`, `name`, `amount`, `category`. 

(For categories with more than one label, just use the first category label in the list)

In [32]:
# Define an empty DataFrame and set the columns as date, name, amount, and category
df = pd.DataFrame(columns = ['date', 'name', 'amount', 'category'])

#create empty list to store data
# Retrieve all the transactions
date = []
name = []
amount = []
category = []

for x in transaction_response['transactions']:
    date.append(x['date']) 
    name.append(x['name'])
    amount.append(x['amount'])
    category.append(x['category'][0])
# Populate the transactions DataFrame with the transactions data
df['date'] = date
df['name'] = name
df['amount'] = amount
df['category'] = category
# Display sample data from the DataFrame
df

,date,name,amount,category
0,2020-07-14,CREDIT CARD 3333 PAYMENT *//,25.00,Payment
1,2020-07-14,Uber 063015 SF**POOL**,5.40,Travel
2,2020-07-13,ACH Electronic CreditGUSTO PAY 123456,5850.00,Transfer
3,2020-07-13,CD DEPOSIT .INITIAL.,1000.00,Transfer
4,2020-07-12,Touchstone Climbing,78.50,Recreation
5,2020-07-12,United Airlines,-500.00,Travel
6,2020-07-11,McDonald's,12.00,Food and Drink
7,2020-07-11,Starbucks,4.33,Food and Drink
8,2020-07-10,SparkFun,89.40,Food and Drink
9,2020-07-09,INTRST PYMNT,-4.22,Transfer


### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [33]:
# Display data types
df.dtypes

date         object
name         object
amount      float64
category     object
dtype: object

In [34]:
# Make any required data type transformation
df['date'] = df['date'].astype('datetime64')
df.dtypes

date        datetime64[ns]
name                object
amount             float64
category            object
dtype: object

In [35]:
# Set the date column as index
df.set_index('date', inplace=True)
# Display sample data
df.head()

,name,amount,category
date,,,
2020-07-14,CREDIT CARD 3333 PAYMENT *//,25.0,Payment
2020-07-14,Uber 063015 SF**POOL**,5.4,Travel
2020-07-13,ACH Electronic CreditGUSTO PAY 123456,5850.0,Transfer
2020-07-13,CD DEPOSIT .INITIAL.,1000.0,Transfer
2020-07-12,Touchstone Climbing,78.5,Recreation


---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:

1. Determine the previous year's gross income and print the results

2. Determine the current monthly income and print the results

3. Determine the projected yearly income and print the results

In [36]:
# Fetch de income data
i = client.Income.get(access_tkn)
# Print the income data
pretty_print_response(i)

ItemError: the requested product is not yet ready. please provide a webhook or try the request again later

In [ ]:
# Determine the previous year's gross income and print the results
gross_income = i['income']['last_year_income_before_tax']
gross_income

In [ ]:
# Determine the current monthly income and print the results
monthly_income = i['income']['income_streams'][0]['monthly_income']
monthly_income

In [ ]:
# Determine the projected yearly income and print the results
projected_yearly_income = i['income']['projected_yearly_income']
projected_yearly_income

---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)

2. Generate a bar chart with the number of transactions for each category

3. Calulate the expenses per month

4. Plot the total expenses per month

### Calculate the expenses per category

In [ ]:
# Compute the total expenses per category
category_total_expenses = df.groupby(['category']).sum()
# Display the expenses per category
category_total_expenses

In [ ]:
# Create a spending categories pie chart.
category_total_expenses.plot.pie(y = 'amount', figsize = (8,8))

### Calculate the expenses per month

In [ ]:
# Create a DataFrame with the total expenses
total_monthly_expenses = df.groupby(pd.Grouper(freq='M')).sum() 

# Display sample data
total_monthly_expenses.head()

In [ ]:
# Create a spending per month bar chart
total_monthly_expenses.plot.bar()
